In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

In [2]:
cities = pd.read_csv('cities.csv',
                     header = None,
                     index_col = 0,
                     names=['x','y'])
cities.head()

,x,y
1,37,52
2,49,49
3,52,64
4,20,26
5,40,30


In [3]:
def gen_rand_sol(df_of_cities):
    rand_seq = np.random.permutation(df_of_cities.index)
    return rand_seq

In [4]:
def calculate_total_distance(df_of_cities, sequence):
    copy_sequence = sequence.copy()
    copy_sequence = np.append(copy_sequence, copy_sequence[0])   #Add first city to the end
    
    original = df_of_cities.loc[copy_sequence]
    shifted = original.iloc[1:].append(original.iloc[0])
    squared = (original.values - shifted.values)**2
    return np.sum(np.sqrt(np.sum(squared, axis=1)))

In [5]:
def cost_with_penalty(df_of_cities, sequence, infeasibility_penalty):
    cost = calculate_total_distance(df_of_cities, sequence)
    if len(sequence) != len(set(sequence)):
        cost += infeasibility_penalty
    
    return cost 

In [6]:
def tournament(df_of_cities, population, tournament_size, infeasibility_penalty):
    pop_size = len(population)
    selected_individuals = []
    for tournament in range(int(pop_size/2)):
        # Pick individuals from population randomly
        candidate_idx = list(np.random.randint(0, pop_size, 2))
        candidate_individuals = population[candidate_idx]
        
        # Calculate cost for for picked individuals, pick the one with the lowest cost
        costs = list(map(lambda sol: cost_with_penalty(df_of_cities, sol, infeasibility_penalty), candidate_individuals))
        winner_idx = np.argmin(costs)
        
        selected_individuals.append(candidate_individuals[winner_idx])
    
    return selected_individuals   

In [7]:
def one_point_co(parents):
    children = []
    for i in range(0, len(parents)-1, 2):
        j = i + 1
        cp = np.random.randint(0, len(parents[0])-1)
        
        first_child = np.append(parents[i][:cp+1], parents[j][cp+1:])
        second_child = np.append(parents[j][:cp+1], parents[i][cp+1:])
        
        children.append(first_child)
        children.append(second_child)
    
    return children

In [8]:
def reciprocal_exchange(children, mutation_prob):
    mutated_children = children.copy()
    idx_list = np.arange(len(children[0]))
    
    for child in mutated_children:
        i,j = np.random.choice(idx_list, 2, replace=False)
        
        if np.random.random() < mutation_prob:
            child[i], child[j] = child[j], child[i]
            
    return mutated_children

In [47]:
def genetic(df_of_cities, pop_size, tournament_size, iteration_number, mutation_prob, infeasibility_penalty):
    population = np.array([gen_rand_sol(df_of_cities) for x in range(pop_size)])
    best_ind = None
    best_cost = np.inf
    
    for iteration in range(iteration_number):
        print(f"\rRunning {iteration+1}", end="")
        parents = tournament(df_of_cities, population, tournament_size, infeasibility_penalty)
        children = one_point_co(parents)
        mutated_children = reciprocal_exchange(children, mutation_prob)
        
        population = np.append(parents, mutated_children, axis=0)
        # Calculate costs for each individual in the population
        costs = list(map(lambda ind: cost_with_penalty(df_of_cities, ind, infeasibility_penalty), population))
        i_best_cost = min(costs)
        i_best_ind = population[np.argmin(costs)]

        if i_best_cost < best_cost and len(i_best_ind) == len(set(i_best_ind)):
            best_cost = i_best_cost
            best_ind = i_best_ind
    
    return best_cost, np.append(best_ind, best_ind[0])

In [ ]:
genetic(cities, 40, 3, 10000, 0.5, 500)

In [40]:
import itertools

def grid_search(function, parameters_dict, repetitions = 1, result_idx_in_return = None):  
    # Create Empty DataFrame with Parameter Names
    column_names = list(parameters_dict.keys()) + ["repetition","result","execution_time"]
    df = pd.DataFrame(columns= column_names)
    
    # Create Search Grid from Dictionary Values
    values = list(parameters_dict.values())
    search_param = list(itertools.product(*values))
    
    # Get the Number of Iterations
    total_iteration = len(search_param) * repetitions
    iterations_done = 0
    
    # Call Function and Record Results
    search_start_time = time.time()
    print(f"Starting {total_iteration} iterations...")
    for iteration, parameters in enumerate(search_param):
        for rep in range(repetitions):
            # Start Timer and Call Function
            start_time = time.time()
            result = function(*parameters)
            exec_time = time.time() - start_time

            df_length = len(df)

            if result_idx_in_return == None:
                # If function returns only one value
                row = parameters + (rep+1, result,exec_time)
            else:
                # If function returns more than one value
                row = parameters + (rep+1, result[result_idx_in_return],exec_time)

            # Append the row into the DataFrame
            df.loc[df_length] = row

            # Print Progress
            iterations_done += 1
            print(f"\rDone #{iterations_done} in {total_iteration}, Time Elapsed: {int(time.time()-search_start_time)} s", end="")
    
    return df

In [90]:
search_dict = {
    "df_of_cities": [cities],
    "pop_size": [20,40],
    "tournament_size": [2,3],
    "iteration_number": [1000, 5000, 10000],
    "mutation_prob": [0.1, 0.5, 0.9],
    "infeasibility_penalty": [500,1000]
}

In [ ]:
results = grid_search(genetic, search_dict, 3, 0)

In [41]:
results.to_excel("Results.xlsx")

In [68]:
results = pd.read_excel('Results.xlsx', index_col=0)
results.drop(labels ='repetition', axis=1, inplace= True)

In [91]:
keys = ['pop_size', 'tournament_size', 'iteration_number', 'mutation_prob', 'infeasibility_penalty']
results.groupby(keys).agg(['mean', 'std','min','max']).sort_values(by=('result','min'))

result  \
                                                                                      mean   
pop_size tournament_size iteration_number mutation_prob infeasibility_penalty                
40       2               10000            0.5           500                     551.308023   
         3               10000            0.1           1000                    551.235577   
         2               10000            0.5           1000                    559.490832   
         3               10000            0.5           500                     546.003016   
20       3               10000            0.5           500                     556.245251   
...                                                                                    ...   
                         1000             0.9           1000                   1201.469537   
                         5000             0.9           1000                   1207.090724   
         2               5000             0.9           1000                   1194.993332   
                         1000             0.9           500                    1162.245124   
         3               10000            0.9           1000                   1302.590924   

                                                                                           \
                                                                                      std   
pop_size tournament_size iteration_number mutation_prob infeasibility_penalty               
40       2               10000            0.5           500                     33.650421   
         3               10000            0.1           1000                    32.903364   
         2               10000            0.5           1000                    36.521053   
         3               10000            0.5           500                     21.997268   
20       3               10000            0.5           500                     31.424569   
...                                                                                   ...   
                         1000             0.9           1000                   252.535737   
                         5000             0.9           1000                   189.632700   
         2               5000             0.9           1000                   197.729244   
                         1000             0.9           500                     72.466968   
         3               10000            0.9           1000                   160.772737   

                                                                                            \
                                                                                       min   
pop_size tournament_size iteration_number mutation_prob infeasibility_penalty                
40       2               10000            0.5           500                     514.468409   
         3               10000            0.1           1000                    516.536453   
         2               10000            0.5           1000                    520.913806   
         3               10000            0.5           500                     521.383953   
20       3               10000            0.5           500                     527.566280   
...                                                                                    ...   
                         1000             0.9           1000                   1023.669901   
                         5000             0.9           1000                   1043.371683   
         2               5000             0.9           1000                   1043.674838   
                         1000             0.9           500                    1100.362360   
         3               10000            0.9           1000                   1153.691237   

                                                                                            \
                                                     

In [83]:
keys = ['pop_size']
results.groupby(keys).agg(['mean', 'std','min','max']).sort_values(by=('result','mean')).drop(['tournament_size','iteration_number','mutation_prob','infeasibility_penalty'], axis=1)

result                                      execution_time  \
                mean         std         min          max           mean   
pop_size                                                                   
40        744.222493  219.671801  514.468409  1486.066616     599.114104   
20        866.336942  286.187612  527.566280  1538.048476     314.978173   

                                               
                 std         min          max  
pop_size                                       
40        417.182033  110.989727  1252.957176  
20        215.179857   61.415360   621.944311

In [82]:
keys = ['tournament_size']
results.groupby(keys).agg(['mean', 'std','min','max']).sort_values(by=('result','mean')).drop(['pop_size','iteration_number','mutation_prob','infeasibility_penalty'], axis=1)

result                                       \
                       mean         std         min          max   
tournament_size                                                    
3                805.375910  266.861758  516.536453  1490.531203   
2                805.744566  258.160221  514.468409  1538.048476   

                execution_time                                      
                          mean         std        min          max  
tournament_size                                                     
3                   456.188087  364.299390  61.415360  1252.957176  
2                   456.592943  357.489959  62.193622  1119.833709

In [81]:
keys = ['iteration_number']
results.groupby(keys).agg(['mean', 'std','min','max']).sort_values(by=('result','mean')).drop(['pop_size','tournament_size','mutation_prob','infeasibility_penalty'], axis=1)

result                                       \
                        mean         std         min          max   
iteration_number                                                    
10000             735.862170  256.140785  514.468409  1497.249297   
5000              798.411555  283.176929  539.829050  1538.048476   
1000              881.356818  226.489949  601.571237  1490.531203   

                 execution_time                                       
                           mean         std         min          max  
iteration_number                                                      
10000                855.771465  273.525051  557.224700  1252.957176  
5000                 430.941068  129.329217  277.101804   579.220094  
1000                  87.584134   24.410239   61.415360   115.137236

In [80]:
keys = ['mutation_prob']
results.groupby(keys).agg(['mean', 'std','min','max']).sort_values(by=('result','mean')).drop(['pop_size','tournament_size','iteration_number','infeasibility_penalty'], axis=1)

result                                       \
                      mean         std         min          max   
mutation_prob                                                     
0.5             673.671320  227.847972  514.468409  1538.048476   
0.1             717.678207  187.057296  516.536453  1497.249297   
0.9            1026.554331  214.936730  678.349113  1490.531203   

              execution_time                                      
                        mean         std        min          max  
mutation_prob                                                     
0.5               457.378968  363.960201  61.587437  1252.957176  
0.1               454.652743  359.022334  61.415360  1138.712222  
0.9               457.166780  362.243239  61.667344  1155.792869

In [78]:
keys = ['infeasibility_penalty']
results.groupby(keys).agg(['mean', 'std','min','max']).sort_values(by=('result','mean')).drop(['pop_size','tournament_size','iteration_number','mutation_prob'], axis=1)

result                                       \
                             mean         std         min          max   
infeasibility_penalty                                                    
500                    785.725951  242.905384  514.468409  1486.066616   
1000                   825.579883  279.508664  516.536453  1538.048476   

                      execution_time                                      
                                mean         std        min          max  
infeasibility_penalty                                                     
500                       455.714233  361.914868  61.587437  1252.957176  
1000                      457.074932  359.861278  61.415360  1156.001355